In [1]:
import pandas as pd
import ast
import numpy as np
from datetime import timedelta

file_path = '2024combined_file.csv'
df = pd.read_csv(file_path)

df['server_received_time'] = pd.to_datetime(df['server_received_time'])
df = df.sort_values(by=['user_id', 'server_received_time'])

df['time_since_last'] = df.groupby('user_id')['server_received_time'].diff()
df['is_new_session'] = df['time_since_last'].isna() | (df['time_since_last'] > pd.Timedelta(minutes=30))
df['session_id'] = df.groupby('user_id')['is_new_session'].cumsum().astype(str) + "_" + df['user_id']
df = df.sort_values(by=['session_id', 'server_received_time'])

df['next_session_time'] = df.groupby('user_id')['server_received_time'].shift(-1)
df['returned'] = (df['next_session_time'] - df['server_received_time']) > pd.Timedelta(days=1)
retention_rates = df.groupby('event_type')['returned'].mean().reset_index()
retention_rates.columns = ['event_type', 'retention_probability']
df = df.merge(retention_rates, on='event_type', how='left')

df['next_action'] = df.groupby('session_id')['event_type'].shift(-1)
df = df.merge(retention_rates, left_on='next_action', right_on='event_type', how='left', suffixes=('', '_next'))
df['y_best'] = df.groupby('event_type')['event_type_next'].transform(
    lambda x: x.mode()[0] if not x.isna().all() else "session_end"
)
df = df[['session_id', 'user_id', 'server_received_time', 'event_type', 'event_properties', 'y_best']]
df['y_best'] = df['y_best'].fillna("session_end")
df = df.sort_values(by=['session_id', 'server_received_time'])

def extract_property(event_json, key):
    try:
        parsed = ast.literal_eval(event_json)
        return parsed.get(key, None)
    except (ValueError, SyntaxError):
        return None

df['event_slug'] = df['event_properties'].apply(lambda x: extract_property(x, 'slug'))
df['event_display_name'] = df['event_properties'].apply(lambda x: extract_property(x, 'displayName'))
df['y_best_detail'] = df.groupby('session_id')['event_slug'].shift(-1)
df['y_best_display'] = df.groupby('session_id')['event_display_name'].shift(-1)
df['y_best'] = df['y_best'].astype(str) + "::" + df['y_best_detail'].astype(str)
df['y_best'] = df['y_best'].fillna(df['event_type'])

generic_actions = ["application-window-opened", "session_start", "session_end", "::None"]
df = df[~df['y_best'].isin(generic_actions)]
df = df[~df['y_best'].str.endswith("::None")]
df = df[~df['y_best'].str.endswith("widget:render")]
df = df[~df['y_best'].str.endswith("widget:render::None")]
df.loc[df['event_type'] == df['y_best'], 'y_best'] = "session_end"
df['y_best'] = df['y_best'].str.replace("::nan", "", regex=False)

df[['primary_y', 'detail_y']] = df['y_best'].str.split("::", n=1, expand=True)
df['detail_y'] = df['detail_y'].fillna("None")

df = df[df['detail_y'] != "None"]

df['session_length'] = df.groupby('session_id')['server_received_time'].transform(
    lambda x: (x.max() - x.min()).total_seconds()
)
df['time_since_last_session'] = df.groupby('user_id')['server_received_time'].diff().dt.total_seconds()
df['total_past_sessions'] = df.groupby('user_id')['session_id'].transform('nunique')

df['event_category'] = df['event_type'].apply(lambda x: x.split(":")[1] if ":" in x else x)
df['hour_of_day'] = df['server_received_time'].dt.hour
df['day_of_week'] = df['server_received_time'].dt.dayofweek  
df['is_working_hours'] = df['hour_of_day'].apply(lambda x: 1 if 9 <= x <= 18 else 0)

df['server_received_time_numeric'] = df['server_received_time'].astype(np.int64) // 10**9

df['previous_event_type'] = df.groupby('session_id')['event_type'].shift(1)
df['time_since_previous_event'] = df.groupby('session_id')['server_received_time_numeric'].diff()
df['session_event_count'] = df.groupby('session_id')['event_type'].transform('count')

df['previous_event_type'].fillna("None", inplace=True)
df['time_since_previous_event'].fillna(0, inplace=True)
df['session_event_count'].fillna(1, inplace=True)

df = df[['session_id'] + list(df.columns.difference(['session_id']))] 

start_date = pd.to_datetime(df['server_received_time_numeric'].min(), unit='s')
cutoff_date = start_date + pd.Timedelta(days=28)
cutoff_numeric = int(cutoff_date.timestamp())
df_first_28 = df[df['server_received_time_numeric'] < cutoff_numeric]

features = [
    'session_length', 'time_since_last_session', 'total_past_sessions',
    'event_type', 'event_category', 'event_slug', 'event_display_name',
    'hour_of_day', 'day_of_week', 'is_working_hours', 'server_received_time_numeric',
    'previous_event_type', 'time_since_previous_event', 'session_event_count'  
]
target = 'y_best'
df = df[features + [target]]

df['time_since_last_session'].fillna(0, inplace=True)
df.loc[df['session_length'] == 0, 'session_length'] = 1
df['total_past_sessions'] = df['total_past_sessions'].fillna(0).astype(int)

print("Total records AFTER Removing 'None':", len(df))
print("Total records in First 28 Days:", len(df_first_28))

X_prepared = df_first_28[features]
y_prepared = df_first_28[target]


Total records AFTER Removing 'None': 325900
Total records in First 28 Days: 289


/var/folders/hc/wd_ljd853ps_jsfw722n74d80000gn/T/ipykernel_54944/1950187504.py:88: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['previous_event_type'].fillna("None", inplace=True)
/var/folders/hc/wd_ljd853ps_jsfw722n74d80000gn/T/ipykernel_54944/1950187504.py:89: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

In [3]:

df_first_28 = df_first_28[['session_id'] + list(df_first_28.columns.difference(['session_id']))]

df_first_28[['primary_y', 'detail_y']] = df_first_28['y_best'].str.split("::", n=1, expand=True)
df_first_28['detail_y'] = df_first_28['detail_y'].fillna("None")

df_first_28 = df_first_28[df_first_28['detail_y'] != "None"]

if 'session_id' not in df_first_28.columns:
    raise KeyError("Error: session_id is missing before computing contextual features!")

print("Checking if session_id exists:", 'session_id' in df_first_28.columns)

df_first_28['previous_event_type'] = df_first_28.groupby('session_id')['event_type'].shift(1)
df_first_28['time_since_previous_event'] = df_first_28.groupby('session_id')['server_received_time'].diff()
df_first_28['session_event_count'] = df_first_28.groupby('session_id')['event_type'].transform('count')

df_first_28['previous_event_type'].fillna("None", inplace=True)
df_first_28['time_since_previous_event'].fillna(0, inplace=True)
df_first_28['session_event_count'].fillna(1, inplace=True)

print("Columns in df_first_28:", df_first_28.columns)


Checking if session_id exists: True
Columns in df_first_28: Index(['session_id', 'day_of_week', 'detail_y', 'event_category',
       'event_display_name', 'event_properties', 'event_slug', 'event_type',
       'hour_of_day', 'is_working_hours', 'previous_event_type', 'primary_y',
       'server_received_time', 'server_received_time_numeric',
       'session_event_count', 'session_length', 'time_since_last_session',
       'time_since_previous_event', 'total_past_sessions', 'user_id', 'y_best',
       'y_best_detail', 'y_best_display'],
      dtype='object')


/var/folders/hc/wd_ljd853ps_jsfw722n74d80000gn/T/ipykernel_54944/2121873437.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_first_28['previous_event_type'].fillna("None", inplace=True)
/var/folders/hc/wd_ljd853ps_jsfw722n74d80000gn/T/ipykernel_54944/2121873437.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are se

In [ ]:

features = [
    'session_length', 'time_since_last_session', 'total_past_sessions',
    'event_type', 'event_category', 'event_slug', 'event_display_name',
    'hour_of_day', 'day_of_week', 'is_working_hours', 'server_received_time_numeric'
]

X = df_first_28[features]
y_primary = df_first_28['primary_y']
y_detail = df_first_28['detail_y']

from sklearn.model_selection import train_test_split

X_train, X_val, y_primary_train, y_primary_val = train_test_split(X, y_primary, test_size=0.2, random_state=42)
y_detail_train = y_detail.loc[X_train.index]  
y_detail_val = y_detail.loc[X_val.index]

print("X_train shape:", X_train.shape)
print("y_primary_train shape:", y_primary_train.shape)
print("y_detail_train shape:", y_detail_train.shape)


X_train shape: (231, 11)
y_primary_train shape: (231,)
y_detail_train shape: (231,)


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

numeric_features = [
    'session_length', 'time_since_last_session', 'total_past_sessions',
    'hour_of_day', 'day_of_week', 'is_working_hours', 'server_received_time_numeric'
]
categorical_features = ['event_type', 'event_category', 'event_slug', 'event_display_name']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)

X_val_transformed = preprocessor.transform(X_val)

print("X_train_transformed shape:", X_train_transformed.shape)
print("X_val_transformed shape:", X_val_transformed.shape)


X_train_transformed shape: (231, 56)
X_val_transformed shape: (58, 56)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.metrics import accuracy_score, classification_report

undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_train_balanced, y_detail_train_balanced = undersampler.fit_resample(X_train_transformed, y_detail_train)


primary_pipeline = Pipeline(steps=[
    ('classifier', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42))
])

detail_pipeline = Pipeline(steps=[
    ('classifier', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42))
])

primary_pipeline.fit(X_train_transformed, y_primary_train)
detail_pipeline.fit(X_train_balanced, y_detail_train_balanced)

y_primary_pred = primary_pipeline.predict(X_val_transformed)
y_detail_pred = detail_pipeline.predict(X_val_transformed)

print("Primary Model Training Accuracy:", primary_pipeline.score(X_train_transformed, y_primary_train))
print("Primary Model Validation Accuracy:", accuracy_score(y_primary_val, y_primary_pred))
print("\nPrimary Model Classification Report:")
print(classification_report(y_primary_val, y_primary_pred))

print("Detail Model Training Accuracy:", detail_pipeline.score(X_train_balanced, y_detail_train_balanced))
print("Detail Model Validation Accuracy:", accuracy_score(y_detail_val, y_detail_pred))
print("\nDetail Model Classification Report:")
print(classification_report(y_detail_val, y_detail_pred))


Primary Model Training Accuracy: 1.0
Primary Model Validation Accuracy: 0.9310344827586207

Primary Model Classification Report:
                                        precision    recall  f1-score   support

                                             1.00      0.80      0.89         5
                               account       1.00      0.88      0.94        17
                         account-lines       0.87      1.00      0.93        13
account-property-rating:pricing-detail       0.00      0.00      0.00         1
                         action-center       0.50      1.00      0.67         2
          action-center:action-details       1.00      1.00      1.00         1
                      agency-dashboard       1.00      1.00      1.00         2
                          all-accounts       1.00      1.00      1.00         2
                     dashboard:my-book       1.00      1.00      1.00        12
         submissions:policy-definition       1.00      1.00      1.00 

/Users/arpitsrivastav/miniconda3/envs/spyder-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/arpitsrivastav/miniconda3/envs/spyder-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/arpitsrivastav/miniconda3/envs/spyder-env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

In [8]:
# Scenario: User is viewing an account summary
sample_input_1 = {
    'session_length': 400,                # Session lasted ~6.7 minutes
    'time_since_last_session': 180,       # 3 minutes since last session
    'total_past_sessions': 15,            # User has had 15 sessions
    'event_type': 'account-lines::account-summary:view',  
    'event_category': 'account-lines',    
    'event_slug': 'account-overview',     # Viewing account details
    'event_display_name': 'view',        
    'hour_of_day': 10,                    # 10 AM
    'day_of_week': 3,                      # Wednesday (assuming Monday is 0)
    'is_working_hours': 1,                 # Within working hours
    'server_received_time_numeric': 1714750000  
}

# Convert to DataFrame
sample_df_1 = pd.DataFrame([sample_input_1])

# Apply preprocessing to match training format
sample_df_transformed_1 = preprocessor.transform(sample_df_1)

# Predict primary and detail components
pred_primary_1 = primary_pipeline.predict(sample_df_transformed_1)
pred_detail_1 = detail_pipeline.predict(sample_df_transformed_1)

# Print predictions
print("Predicted Primary:", pred_primary_1[0])
print("Predicted Detail:", pred_detail_1[0])

# Combine predictions
hierarchical_prediction_1 = pred_primary_1[0] + "::" + pred_detail_1[0]
print("Final Hierarchical Prediction:", hierarchical_prediction_1)


Predicted Primary: account-lines
Predicted Detail: :change-rating-click
Final Hierarchical Prediction: account-lines:::change-rating-click


In [9]:
# Scenario: User is opening a dashboard policy table
sample_input_2 = {
    'session_length': 500,                
    'time_since_last_session': 240,       
    'total_past_sessions': 22,            
    'event_type': 'dashboard:policy-table:render',  
    'event_category': 'dashboard',        
    'event_slug': 'policy-table',         
    'event_display_name': 'render',       
    'hour_of_day': 13,                    
    'day_of_week': 4,                      
    'is_working_hours': 1,                 
    'server_received_time_numeric': 1714800000  
}

# Convert to DataFrame
sample_df_2 = pd.DataFrame([sample_input_2])

# Apply preprocessing
sample_df_transformed_2 = preprocessor.transform(sample_df_2)

# Predict primary and detail components
pred_primary_2 = primary_pipeline.predict(sample_df_transformed_2)
pred_detail_2 = detail_pipeline.predict(sample_df_transformed_2)

# Print predictions
print("Predicted Primary:", pred_primary_2[0])
print("Predicted Detail:", pred_detail_2[0])

# Combine predictions
hierarchical_prediction_2 = pred_primary_2[0] + "::" + pred_detail_2[0]
print("Final Hierarchical Prediction:", hierarchical_prediction_2)


Predicted Primary: account-lines
Predicted Detail: advanced-filters-applied
Final Hierarchical Prediction: account-lines::advanced-filters-applied


In [10]:
# Scenario: User clicks on submission risk insights widget
sample_input_3 = {
    'session_length': 450,                
    'time_since_last_session': 210,       
    'total_past_sessions': 18,            
    'event_type': 'submission:risk-insights:click',  
    'event_category': 'submission',       
    'event_slug': 'risk-dashboard',       
    'event_display_name': 'click',        
    'hour_of_day': 16,                    
    'day_of_week': 2,                      
    'is_working_hours': 1,                 
    'server_received_time_numeric': 1714900000  
}

# Convert to DataFrame
sample_df_3 = pd.DataFrame([sample_input_3])

# Apply preprocessing
sample_df_transformed_3 = preprocessor.transform(sample_df_3)

# Predict primary and detail components
pred_primary_3 = primary_pipeline.predict(sample_df_transformed_3)
pred_detail_3 = detail_pipeline.predict(sample_df_transformed_3)

# Print predictions
print("Predicted Primary:", pred_primary_3[0])
print("Predicted Detail:", pred_detail_3[0])

# Combine predictions
hierarchical_prediction_3 = pred_primary_3[0] + "::" + pred_detail_3[0]
print("Final Hierarchical Prediction:", hierarchical_prediction_3)


Predicted Primary: account-lines
Predicted Detail: :change-rating-click
Final Hierarchical Prediction: account-lines:::change-rating-click


In [14]:
# Scenario: User is downloading a compliance report from account lines
sample_input_7 = {
    'session_length': 300,                
    'time_since_last_session': 100,       
    'total_past_sessions': 20,            
    'event_type': 'account-lines:compliance:download',  
    'event_category': 'account-lines',    
    'event_slug': 'compliance-report',    
    'event_display_name': 'download',     
    'hour_of_day': 15,                    
    'day_of_week': 5,                      
    'is_working_hours': 1,                 
    'server_received_time_numeric': 1715300000  
}

# Convert to DataFrame
sample_df_7 = pd.DataFrame([sample_input_7])

# Apply preprocessing
sample_df_transformed_7 = preprocessor.transform(sample_df_7)

# Predict primary and detail components
pred_primary_7 = primary_pipeline.predict(sample_df_transformed_7)
pred_detail_7 = detail_pipeline.predict(sample_df_transformed_7)

# Print predictions
print("Predicted Primary:", pred_primary_7[0])
print("Predicted Detail:", pred_detail_7[0])

# Combine predictions
hierarchical_prediction_7 = pred_primary_7[0] + "::" + pred_detail_7[0]
print("Final Hierarchical Prediction:", hierarchical_prediction_7)


Predicted Primary: account-lines
Predicted Detail: advanced-filters-applied
Final Hierarchical Prediction: account-lines::advanced-filters-applied


In [ ]:
real_event_type = X_train['event_type'].sample(1).values[0]  # Select a real event type
real_event_category = X_train['event_category'].sample(1).values[0]  # Select a real category

sample_input_fixed = {
    'session_length': 500,
    'time_since_last_session': 240,
    'total_past_sessions': 22,
    'event_type': real_event_type,  
    'event_category': real_event_category, 
    'event_slug': 'policy-table', 
    'event_display_name': 'render',
    'hour_of_day': 13,
    'day_of_week': 4,
    'is_working_hours': 1,
    'server_received_time_numeric': 1714800000  
}

# Convert to DataFrame
sample_df_fixed = pd.DataFrame([sample_input_fixed])

# Apply preprocessing
sample_df_transformed_fixed = preprocessor.transform(sample_df_fixed)

# Predict primary and detail components
pred_primary_fixed = primary_pipeline.predict(sample_df_transformed_fixed)
pred_detail_fixed = detail_pipeline.predict(sample_df_transformed_fixed)

# Print predictions
print("Fixed Input Prediction Primary:", pred_primary_fixed[0])
print("Fixed Input Prediction Detail:", pred_detail_fixed[0])
print("Final Fixed Prediction:", pred_primary_fixed[0] + "::" + pred_detail_fixed[0])


Fixed Input Prediction Primary: dashboard:my-book
Fixed Input Prediction Detail: advanced-filters-applied
Final Fixed Prediction: dashboard:my-book::advanced-filters-applied


In [20]:
real_sample = X_train.sample(3)
real_sample_transformed = preprocessor.transform(real_sample)
pred_primary_real = primary_pipeline.predict(real_sample_transformed)
pred_detail_real = detail_pipeline.predict(real_sample_transformed)

for i in range(3):
    print(f"\nReal Sample {i+1} Prediction:")
    print("Primary:", pred_primary_real[i])
    print("Detail:", pred_detail_real[i])
    print("Final Prediction:", pred_primary_real[i] + "::" + pred_detail_real[i])



Real Sample 1 Prediction:
Primary: dashboard:my-book
Detail: view
Final Prediction: dashboard:my-book::view

Real Sample 2 Prediction:
Primary: dashboard:my-book
Detail: :change-rating-click
Final Prediction: dashboard:my-book:::change-rating-click

Real Sample 3 Prediction:
Primary: goals-and-rules:rules
Detail: :change-rating-click
Final Prediction: goals-and-rules:rules:::change-rating-click
